# HW4. Indexes & Query Processing

## Objectives

In this assignment, you will review what you have learned in the Views and Indexes and Query Processing Modules. you will further practice: 
 - How indexing can change query processing
 - How indexing changes query performance
 - How B-Trees store records
 - Query processing and optimization

## Q1 (10 points): Indexing

In this question, you will be asked to select suitable indexes to speed up query performance and examine the query plan of a SQL query.

We are going to use a new database called flights.db. In the database, there is a single table, called FLIGHTS. The following shows its schema:

    FLIGHTS(fid, year, month_id, day_of_month, day_of_week_id, 
            carrier_id, flight_num, origin_city, origin_state, 
            dest_city, dest_state, departure_delay, taxi_out, 
            arrival_delay, canceled, actual_time, distance)

Note that this task only needs to use four attributes: `carrier_id`, `origin_city`, `actual_time`, and `dest_city`.

In [3]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
%sql sqlite:///flight.db

'Connected: @flight.db'

Consider the following queries:

```sqlite
(a)  SELECT DISTINCT carrier_id
     FROM Flights
     WHERE origin_city = 'Seattle WA' AND actual_time <= 180;
```


```sqlite
(b)  SELECT DISTINCT carrier_id
     FROM Flights
     WHERE origin_city = 'Gunnison CO' AND actual_time <= 180;
```


```sqlite
(c)  SELECT DISTINCT carrier_id
     FROM Flights
     WHERE origin_city = 'Seattle WA' AND actual_time <= 30;
```

Choose one single simple index (index on one attribute) that is most likely to speed up all three queries. Write down the `CREATE INDEX` statement and explain why you chose that index below.

Q1.1. (1 point) What is the CREATE INDEX statement?

In [7]:
%%sql
CREATE INDEX cityIndex ON Flights(origin_city);

 * sqlite:///flight.db
Done.


[]

Q1.2. (1 point) Why did you choose the index? 

The condition of the query is on origin_city and actual_time attributes. While actual time has many differente values, the origin_city has a numebr of specific values (like buckets) where the search could be limited to.

Open a command line shell and start the sqlite program. Connect to the provided flights.db, and check whether the FLIGHTS table has the index that you indicate above. If not, add this index to the FLIGHTS table. 

Q1.3. (0.5 point) Does the FLIGHTS table has the index that you indicate above?

In [11]:
%%sql
SELECT * FROM sqlite_master WHERE type = 'index';

 * sqlite:///flight.db
Done.


type,name,tbl_name,rootpage,sql
index,sqlite_autoindex_FLIGHTS_1,FLIGHTS,33677,None
index,cityIndex,FLIGHTS,10500,CREATE INDEX cityIndex ON Flights(origin_city)


As shows, the FLIGHTS table only has sqlite autoindex. We then created the cityIndex as above, so it has this index now.

Q1.4. (1.5 point) Please check whether each query used the index or not. 

**Hint:** you can use `EXPLAIN QUERY PLAN` to see the query plan of each query. Indicate for each query if it used the index or not. 

In [12]:
%%sql
EXPLAIN QUERY PLAN
SELECT DISTINCT carrier_id
FROM Flights
WHERE origin_city = 'Seattle WA' AND actual_time <= 180;

 * sqlite:///flight.db
Done.


id,parent,notused,detail
4,0,0,SEARCH TABLE Flights USING INDEX cityIndex (origin_city=?)
18,0,0,USE TEMP B-TREE FOR DISTINCT


In [13]:
%%sql
EXPLAIN QUERY PLAN
SELECT DISTINCT carrier_id
FROM Flights
WHERE origin_city = 'Gunnison CO' AND actual_time <= 180;

 * sqlite:///flight.db
Done.


id,parent,notused,detail
4,0,0,SEARCH TABLE Flights USING INDEX cityIndex (origin_city=?)
18,0,0,USE TEMP B-TREE FOR DISTINCT


In [14]:
%%sql
EXPLAIN QUERY PLAN
SELECT DISTINCT carrier_id
FROM Flights
WHERE origin_city = 'Seattle WA' AND actual_time <= 30;

 * sqlite:///flight.db
Done.


id,parent,notused,detail
4,0,0,SEARCH TABLE Flights USING INDEX cityIndex (origin_city=?)
18,0,0,USE TEMP B-TREE FOR DISTINCT


It shows that it did, in all three queries.

Now, consider this query:

```sqlite
(d) SELECT DISTINCT F2.origin_city
     FROM Flights F1, Flights F2
     WHERE F1.dest_city = F2.dest_city
         AND F1.origin_city='Gunnison CO'
         AND F1.actual_time <= 30;
```

Q1.5. (2 points) Choose one simple index (index on one attribute), different from the index for the question above, that is likely to speed up this query. Write down the `CREATE INDEX` statement.

In [15]:
%%sql
CREATE INDEX DestCityIndex ON Flights(dest_city);

 * sqlite:///flight.db
Done.


[]

Check whether the FLIGHTS table has this second index that you indicate above. If not, add this index to the FLIGHTS table. 

Now we want to know how effective the two indexes are. We compare the runtimes of the queries with and without indexes. 

**Hint:** Use `timer on` on sqlite3 command line to turn SQL timer on.

Q1.6. (2 points) Execute queries (a) to (d) on the FLIGHTS table that do not have the two indexes. Please record the runtime of each query. 

`sqlite> .timer on
sqlite> SELECT DISTINCT carrier_id
   ...>      FROM Flights
   ...>      WHERE origin_city = 'Seattle WA' AND actual_time <= 180;
AS
DL
EV
F9
HP
NW
OO
UA
WN
AA
B6
VX
Run Time: real 0.113 user 0.085564 sys 0.027262
sqlite> SELECT DISTINCT carrier_id
   ...>      FROM Flights
   ...>      WHERE origin_city = 'Gunnison CO' AND actual_time <= 180;
OO
Run Time: real 0.108 user 0.081501 sys 0.026101
sqlite> SELECT DISTINCT carrier_id
   ...>      FROM Flights
   ...>      WHERE origin_city = 'Seattle WA' AND actual_time <= 30;
Run Time: real 0.107 user 0.081649 sys 0.025550
sqlite> 
sqlite> SELECT DISTINCT F2.origin_city
   ...>      FROM Flights F1, Flights F2
   ...>      WHERE F1.dest_city = F2.dest_city
   ...>          AND F1.origin_city='Gunnison CO'
   ...>          AND F1.actual_time <= 30;
Run Time: real 0.108 user 0.082539 sys 0.026045`

Q1.7. (2 points) Execute queries (a) to (d) on the FLIGHTS table that has the two indexes. Please record the runtime of each query. 

`sqlite> SELECT DISTINCT carrier_id
   ...> FROM Flights
   ...> WHERE origin_city = 'Seattle WA' AND actual_time <= 180;
AS
DL
EV
F9
HP
NW
OO
UA
WN
AA
B6
VX
Run Time: real 0.017 user 0.010907 sys 0.006287
sqlite> SELECT DISTINCT carrier_id
   ...> FROM Flights
   ...> WHERE origin_city = 'Gunnison CO' AND actual_time <= 180;
OO
Run Time: real 0.000 user 0.000187 sys 0.000173
sqlite> SELECT DISTINCT carrier_id
   ...> FROM Flights
   ...> WHERE origin_city = 'Seattle WA' AND actual_time <= 30;
Run Time: real 0.016 user 0.009144 sys 0.006341
sqlite> SELECT DISTINCT F2.origin_city
   ...>      FROM Flights F1, Flights F2
   ...>      WHERE F1.dest_city = F2.dest_city
   ...>          AND F1.origin_city='Gunnison CO'
   ...>          AND F1.actual_time <= 30;
Run Time: real 0.000 user 0.000155 sys 0.000124`

## Q2 (6 points): B-Trees

Assume:

    (1) blocks can hold either 10 records or 99 keys and 100 pointers
    (2) the average B-tree node is 70% full. This means it will have 69 keys and 70 pointers. 

We can use B-trees as part of several different structures. For each structure described in the questions Q2.1 to Q2.3 below, determine: 

    (a) the total number of blocks needed for a 1,000,000-record file
    (b) the average number of disk I/O’s to retrieve a record given its search key

You may assume nothing is in memory initially, and the search key is the primary key for the records.

Q2.1. (2 points) The data file is a sequential file, sorted on the search key, with 10 records per block. The B-tree is a dense index.

Total Blocks = 114,494 blocks = 100,000 (= 1,000,000/10, for data; @ lowest level) + 14,286 (= 1,000,000/70 @ leaf level) + 204 (= 14,286/70 @ pre-leaf level) + 3 (= 205/70 @ second level) + 1 (the root)


Average Disk Reads - 5

How did we arrive at solution?
First, there are 100,000 data blocks. If there are an average of 70 pointers per block in the
bottom-level nodes of the B-tree, then there are 1,000,000/70 = 14286 B-tree blocks at that
level. The next level of the B-tree requires 1/70th of that, or 204 blocks, and the third level
has 1/70th of that, or 3 blocks. The fourth level has only the root block. The number of
blocks needed is therefore 100,000 + 14,286 + 204 + 3 + 1 = 114,494 blocks.
Since the B-tree has four levels, we must make a total of five disk reads to go through the Btree to the desired data block. 

Q2.2. (2 points) The data file is a sequential file, sorted on the search key, with 10 records per block. The B-tree is a sparse index.

Total Blocks - 101,451 blocks = 100,000 (= 1,000,000/10, for data; @ lowest level) + 1,429 (= 100,000/70 @ leaf level) + 21 (= 1,429/70 @ second level) + 1 (the root)


Average Reads - 4


Q2.3. (2 points) The data file consists of records in no particular order, packed 10 to a block. The B-tree is a dense index.

(a) As we need to keep a pointer for each record we will need 1,000,000 pointers. For full tree (yielding 3-level tree), that will be 10,000 leaf nodes, 100 middle, and 1 root =1000101 blocks in the B-tree (that would be required number of blocks)

If we want to ensure the required blocks with 70% occupancy, assuming each leaf node will have 70 pointers, and we will need 14286 leaf blocks, and level below that would have 205 nodes, and 3 the first level, and 1 node for root. This would total on 14495.

The number of blocks needed for records is 100,0000/10 = 100,000

Total number of blocks: 100,000 + 14495 = 114495

(b) Average Disk Reads - 5


## Q3 (9 points): Query Processing

In the first assignment, given the bank database below:

 - Customer = {<span style="text-decoration:underline">customerID</span>, firstName, lastName, income, birthDate}
 - Account = {<span style="text-decoration:underline">accNumber</span>, type, balance, branchNumber<sup>FK-Branch</sup>}
 - Owns = {<span style="text-decoration:underline">customerID</span><sup>FK-Customer</sup>, <span style="text-decoration:underline">accNumber</span><sup>FK-Account</sup>}
 - Transactions = {<span style="text-decoration:underline">transNumber</span>, <span style="text-decoration:underline">accNumber</span><sup>FK-Account</sup>, amount}
 - Employee = {<span style="text-decoration:underline">sin</span>, firstName, lastName, salary, branchNumber<sup>FK-Branch</sup>}
 - Branch = {<span style="text-decoration:underline">branchNumber</span>, branchName, managerSIN<sup>FK-Employee</sup>, budget}

you wrote a SQL query to:

Show account number, account type, account balance, and transaction amount of the accounts with balance higher than 100,000 and transaction amounts higher than 15000, starting with the accounts with the highest transaction amount and highest account balance. 

Q3.1. (3 points) Parse your query into a query parse tree.

Below is the query we have written for this request previously.

In [ ]:
%%sql
SELECT account.accNumber, account.type, balance, amount 
FROM account join transactions 
WHERE 
    (account.accNumber=transactions.accNumber 
    and transactions.amount>=15000 
    and balance>=100000) 
ORDER BY amount desc, balance desc;

<img src="./ParseTree.png" alt="ParseTree.png" style="width: 650px;"/>

Q3.2. (3 points) Convert your parse tree to the equivalent relational algebraic representation (rewrite if necessary).

<img src="QP.png" alt="QP.png" style="width: 450px;"/> 

Note: The figure is not the rewritten RA. The rewritten one would have the selections pushed down.

Q3.3. (3 points) Assume you have a million records in each of the six tables above. If you need, make necessary assumptions about your storage blocks, as well as about charactristics in the bank.db. Can you enumerate the size and cost of the intermediate tables in your query plan?

T(account) = 1M

T(transactions) = 1M

V(accounts, accNumer) = 1M  (since accNumber is the key for this relation)

**Asumptions:**  ()

V(transactions, accNumber) = 10000

V(transactions,amount) = 3000

V(account, balance) = 1000


**Calculations:**
The following is the required. The rest could be different based on assumptions.

T(transactions JOIN account) = 1Mx1M/MAX(V(transactions,accNumber),V(accounts, accNumer)) = 1Mx1M/300000   

## Submission

Complete the answers to the questions in the [hw4.ipynb](hw4.ipynb) notebook and zip the notebook with additional files that you may have used in a file named HW4.zip, and submit it through Canvas system to your Homework (4) activity.